In [ ]:
import pandas as pd
from collections import Counter
import re
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
nltk_data_path = "D:/nltk_data"  # Change this to your desired directory
if not os.path.exists(nltk_data_path):
    os.makedirs(nltk_data_path)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from kafka import KafkaConsumer, KafkaProducer
# Append the path to NLTK's data search paths
nltk.data.path.append(nltk_data_path)
# Download the required NLTK data to the custom path
nltk.download('punkt', download_dir=nltk_data_path)
nltk.download('stopwords', download_dir=nltk_data_path)
nltk.download('wordnet', download_dir=nltk_data_path)
nltk.download('omw-1.4', download_dir=nltk_data_path)
nltk.data.path.append("D:/nltk_data")

[nltk_data] Downloading package punkt to D:/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to D:/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to D:/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to D:/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
kafka_broker = 'localhost:29092'
consumer = KafkaConsumer(
    "for_ML_data",
    bootstrap_servers=[kafka_broker],
    enable_auto_commit=True,
    value_deserializer=lambda x: x.decode('utf-8')
)
producer = KafkaProducer(
    bootstrap_servers=[kafka_broker],
    linger_ms=5000,
    acks='all',
    max_block_ms=60000
)

In [3]:
# Define preprocessing functions
def clean_text(text):
    """Remove HTML tags, special characters, and punctuation."""
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters and punctuation
    return text

lemmatizer = WordNetLemmatizer()
def lemmatize_tokens(tokens):
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # mapped_tokens = [synonym_mapping.get(token, token) for token in lemmatized_tokens]
    return lemmatized_tokens

def tokenize_text(text):
    tokens = word_tokenize(text, language='english', preserve_line=True)
    return tokens

def handle_missing_data(text, placeholder='Missing'):
    """Handle missing or noisy data."""
    if pd.isnull(text) or text.strip() == "":
        return placeholder
    return text

def to_lowercase(tokens):
    """Convert tokens to lowercase."""
    return [token.lower() for token in tokens]

def remove_stopwords(tokens):
    """Remove stopwords."""
    stop_words = set(stopwords.words('english'))
    return [token for token in tokens if token not in stop_words]

def preprocess_text(text):
    """Complete text preprocessing pipeline."""
    text = handle_missing_data(text)  # Handle missing or noisy data
    # print(f"Handled Missing Data: {text}")
    text = clean_text(text)          # Clean text (remove unwanted characters)
    # print(f"Cleaned Text: {text}")
    tokens = tokenize_text(text)     # Tokenize text
    # print(f"Tokenized Text: {tokens}")
    tokens = to_lowercase(tokens)    # Convert to lowercase
    # print(f"Lowercase Tokens: {tokens}")
    tokens = remove_stopwords(tokens)  # Remove stopwords
    # print(f"Tokens after Removing Stopwords: {tokens}")
    tokens = lemmatize_tokens(tokens)  # Or stem_tokens(tokens) for stemming
    # print(f"Lemmatized and Mapped Tokens: {tokens}")
    return ' '.join(tokens)

In [4]:
df = pd.read_csv('data_cleaned_ml.csv')
df['title'] = df['title'].apply(preprocess_text)
df['categories/keyword'] = df['categories/keyword'].apply(preprocess_text)
df['author_tags'] = df['author_tags'].apply(preprocess_text)

# Save the cleaned DataFrame
#df.to_csv('cleaned_processed.csv', index=False)

In [7]:
exclude_words = {"linking","impact","factor","evaluation","development","associated", "approach","activity","new", "expert",
                    "expression", "application", "case","letter", "research", "paper", "journal", "today", "study", "review","using", 
                    "property","production","update", "based", "change", "applied", "access", "report", "high", "plo", "zootaxa", "novel", "scopus", "outcome", "tev", "performance"
                    ,"effect", "response","journal", "effectiveness", "modified", "systematic", "use", "trial", "state", "acute", "analysis", "association", "characteristic", 
                    "comparison", "composite", "control", "controlled", "different", "process", "acm", "thailand","proceeding", "iop","frontier", "treatment", "model", "conference", "international"
                    ,"series", "science","engineering", "asian", "one", "scientific", "b", "among" }

In [12]:
def predictedModel(input_text, save_df, cat_num, max_df, min_df):
    
    df = save_df
    # Words to exclude
    exclude_words = {"linking","impact","factor","evaluation","development","associated", "approach","activity","new", "expert",
                    "expression", "application", "case","letter", "research", "paper", "journal", "today", "study", "review","using", 
                    "property","production","update", "based", "change", "applied", "access", "report", "high", "plo", "zootaxa", "novel", "scopus", "outcome", "tev", "performance"
                    ,"effect", "response","journal", "effectiveness", "modified", "systematic", "use", "trial", "state", "acute", "analysis", "association", "characteristic", 
                    "comparison", "composite", "control", "controlled", "different", "process", "acm", "thailand","proceeding", "iop","frontier", "treatment", "model", "conference", "international"
                    ,"series", "science","engineering", "asian", "one", "scientific", "b", "among" }

    def preprocess_text(text, exclude_words):
        words = re.findall(r'\b\w+\b', text.lower())  # Tokenize and convert to lowercase
        return [word for word in words if word not in exclude_words]
    # Process the categories column
    all_words_y = []
    for sentence in df['categories/keyword']:
        all_words_y.extend(preprocess_text(sentence, exclude_words))

    # Count the words
    word_counts = Counter(all_words_y)
    # Get the most common words
    most_common_words_y = word_counts.most_common(cat_num)  # Top 10 words
    # Extract words and counts for plotting
    words_y, counts = zip(*most_common_words_y)

    Y_data = list(words_y)
    # Show the plot
    def substitute_row(row, top_words):
        words = preprocess_text(row, exclude_words)
        for word in words:
            if word in top_words:
                return word  # Replace the row with the matched top word
        return row  # Keep original if no top word matches

    df['new_categories'] = df['categories/keyword'].apply(lambda x: substitute_row(x, Y_data))
    def contains_top_word(row, top_words):
    # Tokenize the row text and check for intersection with top words
        words = set(preprocess_text(row, exclude_words))
        return bool(words.intersection(top_words))

    df_Y_data = df[df['categories/keyword'].apply(lambda x: contains_top_word(x, Y_data))].reset_index(drop=True)

    # Get the indices of the rows that match the condition
    matching_indices = df[df['categories/keyword'].apply(lambda x: contains_top_word(x, Y_data))].index

    df_X_data =  df.loc[matching_indices].reset_index(drop=True)

    # Check which rows are being selected (for debugging)
    # print("Rows to be dropped (df_Y_data):")
    # print(len(matching_indices))

    # Get the indices of the rows to drop from the original df
    rows_to_drop = df_Y_data.index

    # Drop the rows from the original df
    df_dropped = df.drop(rows_to_drop).reset_index(drop=True)


    # Recount occurrences of Top 10 words in the filtered DataFrame
    filtered_word_counts = Counter()
    for row in df['categories/keyword']:
        words = preprocess_text(row, exclude_words)
        for word in words:
            if word in Y_data:
                filtered_word_counts[word] += 1

    # Extract words and counts for plotting
    words, counts = zip(*filtered_word_counts.items())

    # Plotting
    # plt.figure(figsize=(10, 6))
    # sns.barplot(x=list(counts), y=list(words), palette="viridis")
    # plt.title("Counts of Top 10 Words in Filtered DataFrame", fontsize=16)
    # plt.xlabel("Count", fontsize=14)
    # plt.ylabel("Words", fontsize=14)
    # plt.tight_layout()

    # Print the filtered DataFrame
    # print("Filtered DataFrame (Only Rows with Top 10 Words):")

    # Show the plot
    # plt.show()


# Instantiate the CountVectorizer
    for category in Y_data:
        df_Y_data[category] = df_Y_data['new_categories'].apply(lambda x: 1 if category in x.split() else 0)

    # Display the updated DataFrame
    df_Y_data.drop(columns=["new_categories", "author_tags","categories/keyword"], inplace=True)

    custom_stop_words = list(set(TfidfVectorizer(stop_words='english').get_stop_words()).union(exclude_words))

    tfidf_vectorizer = TfidfVectorizer(min_df=min_df, stop_words=custom_stop_words, max_df=max_df)
    X_tfidf = tfidf_vectorizer.fit_transform(df_X_data['title'])
    columns2 = tfidf_vectorizer.get_feature_names_out()
    X_df = pd.DataFrame(X_tfidf .toarray(), columns=columns2)


    all_zero_rows = (X_df == 0).all(axis=1) 
# Count rows that have all zeros 
    count_all_zero_rows = all_zero_rows.sum() 
    # print(f"Number of rows with all zeros: {count_all_zero_rows} from {X_df.shape[0]}")

    # List the indices of rows that have all elements equal to zero 
    zero_row_indices = all_zero_rows[all_zero_rows].index.tolist() 
    # print(f"Indices of rows with all zeros: {zero_row_indices}")
    X_input = X_df.drop(zero_row_indices).reset_index(drop=True) 
    Y_output = df_Y_data.drop(zero_row_indices).reset_index(drop=True)
    Y_output = Y_output.drop(columns="title")

    # Assuming X_input and Y_output are already defined
    X = X_input 
    y = Y_output

    # Step 3: Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Step 4: Train the Multi-Label Model using MultiOutputClassifier
    mnb = MultinomialNB()
    multi_target_mnb = MultiOutputClassifier(mnb, n_jobs=-1)
    multi_target_mnb.fit(X_train, y_train)

    # Step 5: Predict on test data
    y_pred = multi_target_mnb.predict(X_test)
    report = classification_report(y_test, y_pred, target_names=Y_data, output_dict=True)
    report = classification_report(y_test, y_pred, target_names=Y_data)
    print(report)

    return multi_target_mnb, tfidf_vectorizer, Y_data
  
    # Step 6: Evaluate the model
    # return classification_report(y_test, y_pred, target_names=Y_data)  # Ensure Y_data matches labels in y_test


In [13]:
predictedModel("", df, 5,0.99, 0.001)

              precision    recall  f1-score   support

    medicine       0.89      0.88      0.88       123
    material       0.72      0.52      0.61       115
      energy       0.77      0.60      0.68       120
      physic       0.77      0.18      0.29        55
  technology       0.90      0.23      0.37       111

   micro avg       0.81      0.53      0.64       524
   macro avg       0.81      0.48      0.57       524
weighted avg       0.81      0.53      0.60       524
 samples avg       0.52      0.53      0.52       524



c:\Users\Asus\anaconda3\envs\dsde-cedt\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Asus\anaconda3\envs\dsde-cedt\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(MultiOutputClassifier(estimator=MultinomialNB(), n_jobs=-1),
 TfidfVectorizer(max_df=0.99, min_df=0.001,
                 stop_words=['do', 'were', 'that', 'approach', 'bill', 'letter',
                             'his', 'about', 'another', 'together', 'several',
                             'under', 'someone', 'which', 'amongst', 'below',
                             'paper', 'composite', 'nobody', 'take', 'towards',
                             'whoever', 'herein', 'have', 'de', 'study',
                             'whether', 'no', 'anywhere', 'co', ...]),
 ['medicine', 'material', 'energy', 'physic', 'technology'])

In [35]:
lasker_df = pd.read_csv('lasker_cleaned_ml.csv')
lasker_df['title'] = lasker_df['title'].apply(preprocess_text)
#lasker_df['categories/keyword'] = df['categories/keyword'].apply(preprocess_text)

nobel_df = pd.read_csv('nobelPrize_cleaned_ml.csv')
nobel_df['title'] = nobel_df['title'].apply(preprocess_text)

In [ ]:
def predict_for_new_data(trained_model, tfidf_vectorizer, Y_data, new_dataset, exclude_words):
    """
    Predict labels for a new dataset with only a 'title' column.

    Parameters:
    - trained_model: The trained multi-label model (MultiOutputClassifier).
    - tfidf_vectorizer: The fitted TfidfVectorizer.
    - Y_data: List of target labels used in training.
    - new_dataset: DataFrame containing the new data to predict. Must have a 'title' column.
    - exclude_words: Set of words to exclude during preprocessing.

    Returns:
    - predictions_df: DataFrame containing the titles and their predicted labels.
    """
    def preprocess_text(text, exclude_words):
        words = re.findall(r'\b\w+\b', text.lower())  # Tokenize and convert to lowercase
        return [word for word in words if word not in exclude_words]

    # Preprocess the 'title' column of the new dataset
    new_dataset['processed_title'] = new_dataset['title'].apply(lambda x: " ".join(preprocess_text(x, exclude_words)))

    # Transform the new dataset using the fitted TfidfVectorizer
    X_new_tfidf = tfidf_vectorizer.transform(new_dataset['processed_title'])

    # Predict labels using the trained model
    y_pred = trained_model.predict(X_new_tfidf)

    # Convert predictions into a DataFrame
    predictions_df = pd.DataFrame(y_pred, columns=Y_data)

    # Add the titles back to the predictions DataFrame
    predictions_df.insert(0, 'title', new_dataset['title'])

    predictions_df.to_csv('prediction.csv', index=False)

    return predictions_df



In [38]:
multi_target_mnb, tfidf_vectorizer, Y_data = predictedModel("", df, 5,0.99, 0.001)

predictions = predict_for_new_data(
    trained_model=multi_target_mnb,
    tfidf_vectorizer=tfidf_vectorizer,
    Y_data=Y_data,
    new_dataset=lasker_df,  # DataFrame with only the 'title' column
    exclude_words=exclude_words
)

              precision    recall  f1-score   support

    medicine       0.89      0.88      0.88       123
    material       0.72      0.52      0.61       115
      energy       0.77      0.60      0.68       120
      physic       0.77      0.18      0.29        55
  technology       0.90      0.23      0.37       111

   micro avg       0.81      0.53      0.64       524
   macro avg       0.81      0.48      0.57       524
weighted avg       0.81      0.53      0.60       524
 samples avg       0.52      0.53      0.52       524



c:\Users\Asus\anaconda3\envs\dsde-cedt\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\Asus\anaconda3\envs\dsde-cedt\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [39]:
predictions = predict_for_new_data(
    trained_model=multi_target_mnb,
    tfidf_vectorizer=tfidf_vectorizer,
    Y_data=Y_data,
    new_dataset=nobel_df,  # DataFrame with only the 'title' column
    exclude_words=exclude_words
)